In [3]:
q = 2
F = GF(q)
C = codes.ReedMullerCode(F, 1, 3)
n = C.length()
V = F^n

In [9]:
C.list()

[(0, 0, 0, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 1),
 (0, 1, 0, 1, 0, 1, 0, 1),
 (1, 0, 1, 0, 1, 0, 1, 0),
 (0, 0, 1, 1, 0, 0, 1, 1),
 (1, 1, 0, 0, 1, 1, 0, 0),
 (0, 1, 1, 0, 0, 1, 1, 0),
 (1, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 1),
 (1, 1, 1, 1, 0, 0, 0, 0),
 (0, 1, 0, 1, 1, 0, 1, 0),
 (1, 0, 1, 0, 0, 1, 0, 1),
 (0, 0, 1, 1, 1, 1, 0, 0),
 (1, 1, 0, 0, 0, 0, 1, 1),
 (0, 1, 1, 0, 1, 0, 0, 1),
 (1, 0, 0, 1, 0, 1, 1, 0)]

In [6]:
def lift_to_subspace(codeword):
    supp = [i for i, x in enumerate(codeword) if x != 0]

    if len(supp) == 0:
        return None
    basis = []
    for i in range(len(supp)):
        vec = vector(F, [1 if j == supp[i] else 0 for j in range(n)])
        basis.append(vec)
    return span(basis)

In [16]:
weight_4_codewords = []
for codeword in C.list():
    w = 0
    for c in codeword:
        if c == 1:
            w += 1
    if w == 4:
        weight_4_codewords.append(codeword)

In [18]:
blocks_from_RM = []
for codeword in weight_4_codewords:
    blocks_from_RM.append(lift_to_subspace(codeword))

In [ ]:
def is_subspace_design(blocks, t, V):
    t_subspaces = list(V.subspaces(t))
    res = True
    

In [57]:
from itertools import product

def projective_reed_muller_code(q, m, d):
    F = GF(q)
    PG = ProjectiveSpace(m, F)

    points = PG.rational_points()

    normalized_points = []
    for p in points:
        coords = list(p)
        for a in coords:
            if a != F(0):
                inv = 1 / a
                normalized = [inv * x for x in coords]
                break
        normalized_points.append(tuple(normalized))
    
    R = PolynomialRing(F, 'x', m+1)
    n = R.ngens()

    monomials = []
    for exponents in product(*[range(d+1) for i in range(n)]):
        if sum(exponents) == d:
            mon = prod(R.gen(i)**exponents[i] for i in range(n))
            monomials.append(mon)
    rows = []
    for mon in monomials:
        row = [mon(*pt) for pt in normalized_points]
        rows.append(row)
    G = Matrix(F, rows)

    C = LinearCode(G)
    return C

PRMC = projective_reed_muller_code(2, 2, 2)

In [38]:
def codeword_to_matrix(codeword, m):
    expanded = []
    for element in codeword:
        poly = element.polynomial()
        coeffs = poly.coefficients(sparse=False)
        coeffs += [0] * (m - len(coeffs))
        expanded.append(coeffs)
    flat_list = [coeff for vec in expanded for coeff in vec]
    return Matrix(GF(2), len(codeword), m, flat_list)

In [39]:
matrix_codewords = []
for codeword in PRMC:
    matrix = codeword_to_matrix(codeword, 2)
    matrix_codewords.append(matrix)

In [60]:
from itertools import product

def projective_reed_muller_code(q, m, d):
    F = GF(q)
    PG = ProjectiveSpace(m, F)
    points = PG.rational_points()

    normalized_points = []
    for p in points:
        coords = list(p)
        for a in coords:
            if a != F(0):
                inv = 1 / a
                normalized = [inv * x for x in coords]
                break
        normalized_points.append(tuple(normalized))

    R = PolynomialRing(F, 'x', m+1)
    n_vars = R.ngens()

    monomials = []
    for exponents in product(*[range(d+1) for i in range(n_vars)]):
        if sum(exponents) == d:
            mon = prod(R.gen(i)**exponents[i] for i in range(n_vars))
            monomials.append(mon)

    rows = []
    for mon in monomials:
        row = [mon(*pt) for pt in normalized_points]
        rows.append(row)
    G = Matrix(F, rows)
    
    C = LinearCode(G)
    return C, normalized_points

In [61]:
def support_projective_span(c, normalized_points, F):
    support_indices = [i for i, val in enumerate(c) if val != F(0)]
    support_points = [normalized_points[i] for i in support_indices]

    vecs = [vector(F, pt) for pt in support_points]
    if vecs:
        V = span(vecs)
        proj_dim = V.dimension() - 1  
    else:
        proj_dim = -1
        
    return set(support_points), proj_dim

In [62]:
def PG_subspaces(PG, k):
    F = PG.base_ring()
    m = PG.dimension()
    V = VectorSpace(F, m+1)
    return V.subspaces(k+1)

In [63]:
def subspaces_in_support(c, normalized_points, PG, k):
    F = PG.base_ring()
    support, _ = support_projective_span(c, normalized_points, F)
    support_subspaces = []

    for U in PG_subspaces(PG, k):
        U_points = {pt for pt in normalized_points if vector(F, pt) in U}
        if U_points.issubset(support):
            support_subspaces.append(U)
    return support_subspaces

In [64]:
def max_subspace_in_support(c, normalized_points, PG):
    F = PG.base_ring()
    subspaces_dict = {}
    max_k = -1
    for k in range(PG.dimension()+1):
        subs_k = subspaces_in_support(c, normalized_points, PG, k)
        if subs_k:
            subspaces_dict[k] = subs_k
            max_k = k
    return max_k, subspaces_dict

In [72]:
q = 4; m = 2; d = 2
F = GF(q)
C, norm_pts = projective_reed_muller_code(q, m, d)
PG = ProjectiveSpace(m, F)

In [74]:
PG

Projective Space of dimension 2 over Finite Field in z2 of size 2^2